In [3]:
GENIUS_API_TOKEN = "xjsAbJo93Z6B9W7tLL7WVKICkPJB8roxefG6jSlzFEOh2zM1pFVG6Sy2Yz6QZa07"
import requests
from bs4 import BeautifulSoup
import os
import re
import pandas as pd

In [4]:
# get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

# get Genius.com song url's from artist object
def request_song_info(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                song_artist = song['result']['primary_artist']['name']
                song_title = song['result']['title']
                songs.append([song_title, song_artist, url])
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs #2d list

In [5]:
def get_text(elements):
    text = ''
    for c in elements:
        for t in c.select('a, span'):
            t.unwrap()
        if c:
            c.smooth()
            text += c.get_text(strip=True, separator='\n')
    return text

# Get song lyrics based on url
def scrape_song_lyrics(url):
    text = ''
    soup = BeautifulSoup(requests.get(url).content, 'html')
    
    cs = soup.select('div[class^="Lyrics__Container"]')
    if cs:
        text = get_text(cs)
    else:
        text = get_text(soup.select('.lyrics'))
    return text

In [14]:
def write_lyrics_to_file(artist_name, song_count):
    #f = open('lyrics/' + artist_name.lower() + '.txt', 'wb')
    header = ['Song Title', 'Artist', 'Lyrics']
    song_info_list = []
    

    songs_info = request_song_info(artist_name, song_count)
    for info in songs_info:
        #put in get song and artist
        song_title = info[0]
        artist = info[1]
        url = info[2]
        lyrics = scrape_song_lyrics(url)
        song_info_list.append([song_title, artist, lyrics])
        
        
        #lyrics = lyrics.replace('[', '\n[')
        #f.write(lyrics.encode("utf8"))
    #f.close()
    song_info_df = pd.DataFrame(song_info_list, columns=header)
    song_info_df.to_csv('lyrics/{}.csv'.format(artist_name), index=False)

    num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + '.csv', 'rb'))
    print('Wrote {} lines to file from {} songs. Wrote to lyrics/{}.csv'.format(num_lines, song_count, artist_name))
  
# DEMO  
#write_lyrics_to_file('Kanye West', 100)

Found 1 songs by Kanye West
Wrote 315 lines to file from 1 songs. Wrote to lyrics/Kanye West.csv


In [21]:
df = pd.read_csv("artists.csv")
artist_list = df['Artist'].to_list()

for artist in artist_list:
    

    if os.path.exists(f"lyrics/{artist}.csv") or artist == 'Tyler the Creator':
        continue
    else:
        print("Reading and Saving {} lyrics".format(artist))
        write_lyrics_to_file(artist, 100)
    print("")